In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from keras.callbacks import ModelCheckpoint


#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


Attempt # 1: Remove Status column

In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(["EIN","NAME"],axis=1, inplace=True)

# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [3]:
# Look at APPLICATION_TYPE value counts for binning
application_df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [4]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

app_vc = application_df.APPLICATION_TYPE.value_counts()
application_types_to_replace = app_vc[app_vc < 500].index

# # Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# # Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Look at CLASSIFICATION value counts for binning
application_df.CLASSIFICATION.value_counts()


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2600        1
C1732        1
C2380        1
C2150        1
C8210        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [6]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
class_vc = application_df.CLASSIFICATION.value_counts()
classifications_to_replace = class_vc[class_vc < 1800].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [7]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies = pd.get_dummies(application_df)
application_dummies.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [8]:
# Split our preprocessed data into our features and target arrays
y = application_dummies["IS_SUCCESSFUL"].values
X = application_dummies.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78) 

In [9]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 80)                3520      
_________________________________________________________________
dense_1 (Dense)              (None, 30)                2430      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 31        
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]) 

# Train the model
# checkpoint = ModelCheckpoint("AlphabetSoupCharity_Optimization.hdf5", monitor='loss', verbose=1, mode='auto', period=5)
# fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[checkpoint])

fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 404us/step - loss: 0.5721 - accuracy: 0.7198
Epoch 2/100
804/804 [==============================] - 0s 413us/step - loss: 0.5557 - accuracy: 0.7302
Epoch 3/100
804/804 [==============================] - 0s 416us/step - loss: 0.5531 - accuracy: 0.7307
Epoch 4/100
804/804 [==============================] - 0s 408us/step - loss: 0.5511 - accuracy: 0.7327
Epoch 5/100
804/804 [==============================] - 0s 415us/step - loss: 0.5494 - accuracy: 0.7340
Epoch 6/100
804/804 [==============================] - 0s 431us/step - loss: 0.5488 - accuracy: 0.7329
Epoch 7/100
804/804 [==============================] - 0s 427us/step - loss: 0.5482 - accuracy: 0.7345
Epoch 8/100
804/804 [==============================] - 0s 422us/step - loss: 0.5476 - accuracy: 0.7341
Epoch 9/100
804/804 [==============================] - 0s 427us/step - loss: 0.5471 - accuracy: 0.7342
Epoch 10/100
804/804 [==============================] - 0s 431us/step - l

In [12]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5557 - accuracy: 0.7245
Loss: 0.5556736588478088, Accuracy: 0.7245481014251709


Attempt # 1 result: Accuracy of approximately 72% with approximate loss of 55%.

Attempt #2: 

In [13]:
#  Import and read the charity_data.csv.
import pandas as pd 
application_df2 = pd.read_csv("charity_data.csv")

# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df2.drop(["EIN","NAME","STATUS"],axis=1, inplace=True)

# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

app_vc2 = application_df2.APPLICATION_TYPE.value_counts()
application_types_to_replace2 = app_vc2[app_vc2 < 500].index

# # Replace in dataframe
for app in application_types_to_replace2:
    application_df2['APPLICATION_TYPE'] = application_df2['APPLICATION_TYPE'].replace(app,"Other")

# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
class_vc2 = application_df2.CLASSIFICATION.value_counts()
classifications_to_replace2 = class_vc2[class_vc2 < 1800].index

# Replace in dataframe
for cls in classifications_to_replace2:
    application_df2['CLASSIFICATION'] = application_df2['CLASSIFICATION'].replace(cls,"Other")

# Convert categorical data to numeric with `pd.get_dummies`
application_dummies2 = pd.get_dummies(application_df2)

# Split our preprocessed data into our features and target arrays
y2 = application_dummies2["IS_SUCCESSFUL"].values
X2 = application_dummies2.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, random_state=78) 

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X2_scaler = scaler.fit(X2_train)

# Scale the data
X2_train_scaled = X2_scaler.transform(X2_train)
X2_test_scaled = X2_scaler.transform(X2_test)

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features2 = len(X2_train[0])
hidden_nodes_layer1_2 =  60
hidden_nodes_layer2_2 = 20

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1_2, input_dim=number_input_features2, activation="relu")
)

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2_2, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]) 

# Train the model
# checkpoint = ModelCheckpoint("AlphabetSoupCharity_Optimization.hdf5", monitor='loss', verbose=1, mode='auto', period=5)
# fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[checkpoint])

fit_model = nn2.fit(X2_train_scaled,y2_train,epochs=100)

# Evaluate the model using the test data
model_loss2, model_accuracy2 = nn2.evaluate(X2_test_scaled,y2_test,verbose=2)
print(f"Loss: {model_loss2}, Accuracy: {model_accuracy2}")

Epoch 1/100
804/804 [==============================] - 1s 428us/step - loss: 1.4307 - accuracy: 0.6765
Epoch 2/100
804/804 [==============================] - 0s 435us/step - loss: 0.7605 - accuracy: 0.7136
Epoch 3/100
804/804 [==============================] - 0s 434us/step - loss: 0.6968 - accuracy: 0.7223
Epoch 4/100
804/804 [==============================] - 0s 436us/step - loss: 0.6771 - accuracy: 0.7208
Epoch 5/100
804/804 [==============================] - 0s 435us/step - loss: 0.6981 - accuracy: 0.7226
Epoch 6/100
804/804 [==============================] - 0s 436us/step - loss: 0.6599 - accuracy: 0.7291
Epoch 7/100
804/804 [==============================] - 0s 427us/step - loss: 0.6695 - accuracy: 0.7255
Epoch 8/100
804/804 [==============================] - 0s 434us/step - loss: 0.6483 - accuracy: 0.7271
Epoch 9/100
804/804 [==============================] - 0s 437us/step - loss: 0.6070 - accuracy: 0.7294
Epoch 10/100
804/804 [==============================] - 0s 448us/step - l

Attempt 2 summary:

Changes made: Additionally remove "STATUS" column. Reduce number of nodes from 80/30 to 60/20.
Results: Accuracy of approximately 67% with approximate loss of 62%.

(Attempt # 1 result: Accuracy of approximately 72% with approximate loss of 55%.)

Analysis of changes:
With the lower accuracy rate as well as the lower confidence in the model's predictions, it looks like the new changes caused more errors, and the next attempt needs to be in an opposite direction than the one taken here.

Attempt 3:

In [15]:
#  Import and read the charity_data.csv.
import pandas as pd 
application_df_3 = pd.read_csv("charity_data.csv")

# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df_3.drop(["EIN","NAME","STATUS"],axis=1, inplace=True)

# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

app_vc_3 = application_df_3.APPLICATION_TYPE.value_counts()
application_types_to_replace_3 = app_vc_3[app_vc_3 < 500].index

# # Replace in dataframe
for app in application_types_to_replace_3:
    application_df_3['APPLICATION_TYPE'] = application_df_3['APPLICATION_TYPE'].replace(app,"Other")

# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
class_vc_3 = application_df_3.CLASSIFICATION.value_counts()
classifications_to_replace_3 = class_vc_3[class_vc_3 < 1800].index

# Replace in dataframe
for cls in classifications_to_replace_3:
    application_df_3['CLASSIFICATION'] = application_df_3['CLASSIFICATION'].replace(cls,"Other")

# Convert categorical data to numeric with `pd.get_dummies`
application_dummies_3 = pd.get_dummies(application_df_3)

# Split our preprocessed data into our features and target arrays
y3 = application_dummies_3["IS_SUCCESSFUL"].values
X3 = application_dummies_3.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, random_state=78) 

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler3 = scaler.fit(X3_train)

# Scale the data
X_train_scaled3 = X_scaler3.transform(X3_train)
X_test_scaled3 = X_scaler3.transform(X3_test)

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features_3 = len(X3_train[0])
hidden_nodes_layer1_3 =  80
hidden_nodes_layer2_3 = 35
hidden_nodes_layer3_3 = 5


nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1_3, input_dim=number_input_features_3, activation="relu")
)

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2_3, activation="relu"))

# Third hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer3_3, activation="relu"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]) 

# Train the model
checkpoint3 = ModelCheckpoint("AlphabetSoupCharity_Optimization.hdf5", monitor='loss', verbose=1, mode='auto', period=5)
fit_model = nn3.fit(X_train_scaled3,y3_train,epochs=100,callbacks=[checkpoint3])

# Evaluate the model using the test data
model_loss3, model_accuracy3 = nn3.evaluate(X_test_scaled3,y3_test,verbose=2)
print(f"Loss: {model_loss3}, Accuracy: {model_accuracy3}")

Epoch 1/100
804/804 [==============================] - 1s 467us/step - loss: 0.8467 - accuracy: 0.7006
Epoch 2/100
804/804 [==============================] - 0s 460us/step - loss: 0.6704 - accuracy: 0.7182
Epoch 3/100
804/804 [==============================] - 0s 464us/step - loss: 0.6408 - accuracy: 0.7228
Epoch 4/100
804/804 [==============================] - 0s 461us/step - loss: 0.6120 - accuracy: 0.7230
Epoch 5/100
804/804 [==============================] - 0s 457us/step - loss: 0.5855 - accuracy: 0.7211

Epoch 00005: saving model to AlphabetSoupCharity_Optimization.hdf5
Epoch 6/100
804/804 [==============================] - 0s 467us/step - loss: 0.5904 - accuracy: 0.7191
Epoch 7/100
804/804 [==============================] - 0s 463us/step - loss: 0.5653 - accuracy: 0.7292
Epoch 8/100
804/804 [==============================] - 0s 450us/step - loss: 0.5702 - accuracy: 0.7264
Epoch 9/100
804/804 [==============================] - 0s 460us/step - loss: 0.5692 - accuracy: 0.7312
Epoch

Attempt 3 summary:

Changes made: Additionally remove "STATUS" column. Added a third hidden layer, and changed number of nodes from 80/30 to 80/35/5.

Results: Accuraccy 72.6% with loss rate of 58%

(Attempt # 1 result: Accuracy of approximately 72% with approximate loss of 55%.)
(Attempt # 2 result: Accuracy of approximately 67% with approximate loss of 62%.)

Analysis of changes:
The changes made in this attempt resulted in both a higher accuracy, though the confidence level dropped by about 3%. This could indicate that changes are moving in the right direction, though better results are still achievable. From this step, further actions to be considered include changing the number of nodes per layer, possibly adding more nodes, or adding more data for a larger training sample.